In [69]:
print("Welcome to fertilizer Prediction")

Welcome to fertilizer Prediction


In [97]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


In [98]:
df=pd.read_csv("/content/Fertilizer Prediction.csv")

In [99]:
df.head()

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,32,51,41,Red,Ground Nuts,7,3,19,14-35-14
1,35,58,35,Black,Cotton,4,14,16,Urea
2,27,55,43,Sandy,Sugarcane,28,0,17,20-20
3,33,56,56,Loamy,Ground Nuts,37,5,24,28-28
4,32,70,60,Red,Ground Nuts,4,6,9,14-35-14


In [101]:
final=df.rename(columns={"Temparature":"temperature","Humidity":"humidity","Moisture":"soil_moisture"})
final.head()


,temperature,humidity,soil_moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,32,51,41,Red,Ground Nuts,7,3,19,14-35-14
1,35,58,35,Black,Cotton,4,14,16,Urea
2,27,55,43,Sandy,Sugarcane,28,0,17,20-20
3,33,56,56,Loamy,Ground Nuts,37,5,24,28-28
4,32,70,60,Red,Ground Nuts,4,6,9,14-35-14


In [120]:
final['Fertilizer Name'].unique()
final['Fertilizer Name'].value_counts()



,count
Fertilizer Name,
14-35-14,14492
10-26-26,14378
Urea,14325
28-28,14232
DAP,14220
20-20,14181
17-17-17,14172


In [102]:
final.columns

Index(['temperature', 'humidity', 'soil_moisture', 'Soil Type', 'Crop Type',
       'Nitrogen', 'Potassium', 'Phosphorous', 'Fertilizer Name'],
      dtype='object')

In [105]:

cat_features = [X.columns.get_loc(col) for col in ['Crop Type', 'Soil Type']]

X = final[['temperature','humidity','soil_moisture','Crop Type','Soil Type']]
y = final['Fertilizer Name']

In [109]:

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42,stratify=y)

model = CatBoostClassifier(iterations=200,
    depth=8,
    learning_rate=0.08,
    loss_function='MultiClass',
    eval_metric='MultiClass',
    cat_features=cat_features,
    random_seed=42,
    verbose=100)
model.fit(x_train, y_train)

0:	learn: 1.9457936	total: 2.31s	remaining: 7m 40s
100:	learn: 1.9366149	total: 1m 9s	remaining: 1m 7s
199:	learn: 1.9127331	total: 2m 33s	remaining: 0us


In [123]:
import numpy as np
"""

proba = model.predict_proba(x_test)

class_names = model.classes_

K = 6
topk_indices = np.argsort(proba, axis=1)[:, -K:][:, ::-1]

topk_fertilizers = class_names[topk_indices]

topk_probs = np.take_along_axis(proba, topk_indices, axis=1)

i = 0
for rank in range(K):
    print(f"{rank+1}. {topk_fertilizers[i][rank]} → {topk_probs[i][rank]:.3f}")

y_true = y_test.values

topk_accuracy = np.mean([
    y_true[i] in topk_fertilizers[i]
    for i in range(len(y_true))
])

print("Top-K Accuracy:", topk_accuracy)"""


temperature = float(input("Enter temperature (°C): "))
humidity = float(input("Enter humidity (%): "))
soil_moisture = float(input("Enter soil moisture (%): "))
crop_type = input("Enter crop type: ")
soil_type = input("Enter soil type: ")
sample_input = pd.DataFrame([{
    "temperature": temperature,
    "humidity": humidity,
    "soil_moisture": soil_moisture,
    "Crop Type": crop_type,
    "Soil Type": soil_type
}])

proba = model.predict_proba(sample_input)

class_names = model.classes_

K = 3

topk_idx = np.argsort(proba, axis=1)[:, -K:][:, ::-1]
topk_fertilizers = class_names[topk_idx]
topk_probs = np.take_along_axis(proba, topk_idx, axis=1)
print("Recommended Fertilizers:")
for i in range(K):
    print(f"{i+1}. {topk_fertilizers[0][i]} → {topk_probs[0][i]*100:.1f}% suitability")



Enter temperature (°C): 29
Enter humidity (%): 68
Enter soil moisture (%): 42
Enter crop type: Rice
Enter soil type: Loamy
Recommended Fertilizers:
1. 20-20 → 29.8% suitability
2. DAP → 16.0% suitability
3. 14-35-14 → 13.9% suitability
